<a href="https://colab.research.google.com/github/shin584/project/blob/main/Multi_Cas_10_Head_Hydra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive

# 1. 구글 드라이브 마운트 (이 코드를 실행하면 팝업창이 뜨고 승인해야 합니다)
drive.mount('/content/drive')

# 2. 파일 경로 설정
# '내 드라이브'는 Colab에서 '/content/drive/MyDrive'로 인식됩니다.
base_path = '/content/drive/MyDrive/Colab Notebooks/data'
file_name = 'kim_2020_table_s8.xlsx' # 파일명이 정확한지 확인하세요!

# 전체 경로 합치기
file_path = os.path.join(base_path, file_name)

# 3. 잘 연결됐는지 확인
if os.path.exists(file_path):
    print(f"✅ 파일을 찾았습니다! 경로: {file_path}")
else:
    print(f"❌ 파일을 찾을 수 없습니다. 경로를 다시 확인해주세요: {file_path}")
    print("팁: 폴더명이나 파일명에 띄어쓰기가 있는지, 대소문자가 맞는지 확인해보세요.")

Mounted at /content/drive
✅ 파일을 찾았습니다! 경로: /content/drive/MyDrive/Colab Notebooks/data/kim_2020_table_s8.xlsx


In [5]:
import pandas as pd
import numpy as np


# ---------------------------------------------------------
# 1. 원-핫 인코딩 함수 (길이 보정 기능 추가)
# ---------------------------------------------------------
def universal_one_hot(sequence, target_length=34):
    mapping = {'A': [1,0,0,0], 'C': [0,1,0,0], 'G': [0,0,1,0], 'T': [0,0,0,1]}
    sequence = sequence.upper()

    # 1. 기본 인코딩
    arr = [mapping.get(base, [0,0,0,0]) for base in sequence]

    # 2. 패딩 (Padding): 모자란 길이만큼 [0,0,0,0] 추가
    current_len = len(arr)
    if current_len < target_length:
        pad_len = target_length - current_len
        # 뒤쪽에 0으로 채움 (Zero-padding)
        for _ in range(pad_len):
            arr.append([0,0,0,0])

    return np.array(arr)

# ---------------------------------------------------------
# 2. 데이터 로드 및 통합
# ---------------------------------------------------------
def load_universal_dataset(spcas9_path, cas12a_path):
    print("🔄 데이터 통합 중... (Universal Loading)")

    X_list = []
    Y_list = []      # 점수 값
    Type_list = []   # 0: SpCas9, 1: Cas12a

    # --- [A] SpCas9 데이터 로드 ---
    # (여기는 잘 되셨다고 했으니 그대로 둡니다)
    df_sp = pd.read_excel(spcas9_path)
    try:
        sp_seq_col = [c for c in df_sp.columns if 'Input sequence' in c][0]
    except IndexError:
        print("❌ SpCas9 컬럼 에러")
        return None, None, None

    sp_targets = [
        'SpCas9', 'SpCas9-NG', 'VRQR variant', 'xCas', 'Sniper-Cas9',
        'SpCas9-HF.1', 'eSpCas9(1.1)', 'HypaCas9', 'evoCas9'
    ]

    print(f"   📂 SpCas9 데이터 처리 중... ({len(df_sp)}개)")
    for _, row in df_sp.iterrows():
        seq = row[sp_seq_col]
        if not isinstance(seq, str) or len(seq) != 30: continue
        X_list.append(universal_one_hot(seq, target_length=34))

        scores = []
        for t in sp_targets:
            val = pd.to_numeric(row.get(t, 0), errors='coerce')
            if np.isnan(val): val = 0.0
            scores.append(val)
        scores.append(0.0)
        Y_list.append(scores)
        Type_list.append(0)

    # --- [B] Cas12a 데이터 로드 (문제 해결 파트) ---
    df_cpf1 = pd.read_excel(cas12a_path) # header 옵션 없음 (기본값)

    # 1. 서열 컬럼 찾기 (34 bp)
    try:
        cpf1_seq_col = [c for c in df_cpf1.columns if '34 bp' in c][0]
    except IndexError:
        print("❌ 에러: '34 bp' 서열 컬럼을 못 찾았습니다.")
        print(f"   (현재 컬럼 목록: {df_cpf1.columns.tolist()})")
        return None, None, None

    # 2. 점수 컬럼 찾기 (오타 무시하고 핵심 단어로 찾기)
    # 'Indel'과 'Background' 글자가 들어간 컬럼은 무조건 가져옵니다.
    # (subtracted 철자가 틀려도 찾을 수 있음)
    candidates = [c for c in df_cpf1.columns if 'Indel' in c and 'Background' in c]

    if len(candidates) > 0:
        cpf1_score_col = candidates[0] # 첫 번째 발견된 것 사용
    else:
        print("❌ 에러: 점수 컬럼을 못 찾았습니다.")
        print(f"   (현재 컬럼 목록: {df_cpf1.columns.tolist()})")
        return None, None, None

    print(f"   📂 Cas12a 데이터 처리 중... ({len(df_cpf1)}개)")
    print(f"      👉 매핑된 점수 컬럼명: '{cpf1_score_col}' (오타가 있어도 OK)")

    for _, row in df_cpf1.iterrows():
        seq = row[cpf1_seq_col]
        if not isinstance(seq, str) or len(seq) < 34: continue

        X_list.append(universal_one_hot(seq[:34], target_length=34))

        score = pd.to_numeric(row[cpf1_score_col], errors='coerce')
        if np.isnan(score): score = 0.0

        scores = [0.0] * len(sp_targets) + [score]
        Y_list.append(scores)
        Type_list.append(1)

    return np.array(X_list), np.array(Y_list), np.array(Type_list)

# ---------------------------------------------------------
# 실행 예시 (경로만 수정해서 쓰세요)
# ---------------------------------------------------------
sp_path = '/content/drive/MyDrive/Colab Notebooks/data/kim_2020_table_s8.xlsx'
cpf1_path = '/content/drive/MyDrive/Colab Notebooks/data/kim_2018_cas12a_dataset.xlsx'
X, Y, Types = load_universal_dataset(sp_path, cpf1_path)

print(f"✅ 통합 완료! X shape: {X.shape}") # (N, 34, 4)가 나와야 함



🔄 데이터 통합 중... (Universal Loading)
   📂 SpCas9 데이터 처리 중... (40128개)
   📂 Cas12a 데이터 처리 중... (15002개)
      👉 매핑된 점수 컬럼명: 'Indel frequency
(%, Background)' (오타가 있어도 OK)
✅ 통합 완료! X shape: (55128, 34, 4)


In [ ]:
!pip install ViennaRNA

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 58.5 MB/s eta 0:00:00


In [10]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Conv1D, LSTM, Dense, Concatenate, BatchNormalization, Dropout, Bidirectional, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import re

def build_universal_hydra_model(target_names):
    """
    SpCas9 계열(30bp)과 Cas12a(34bp)를 동시에 예측하는 Universal Hydra Model
    - Input Shape: (34, 4)로 통합
    - Architecture: Shared Conv1D + Bidirectional LSTM -> 10 Separate Heads
    """

    # ---------------------------------------------------------
    # 1. 입력층 정의 (Inputs)
    # ---------------------------------------------------------
    # 핵심 변경: 입력 길이가 34bp로 늘어남
    input_seq = Input(shape=(34, 4), name='Input_Sequence')

    # 물리적 특성 및 효율 보정값 (없으면 0으로 입력됨)
    input_energy = Input(shape=(1,), name='Input_Energy')
    input_trna = Input(shape=(1,), name='Input_tRNA')

    # ---------------------------------------------------------
    # 2. Shared Body (공유 신경망)
    # ---------------------------------------------------------
    # [Step A] Convolution Layer: 지역적 특징(Motif) 추출
    # 필터 수 128, 커널 크기 5 (약 5bp 단위의 패턴 인식)
    x = Conv1D(filters=128, kernel_size=5, activation='relu', padding='same')(input_seq)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x) # 과적합 방지

    # [Step B] Recurrent Layer: 서열의 문맥(Context) 학습
    # 🌟 Bidirectional 추가: Cas12a는 앞쪽(5')이 중요하고, SpCas9은 뒤쪽(3')이 중요함.
    # 양방향으로 훑어서 두 특징을 모두 놓치지 않도록 설계.
    x = Bidirectional(LSTM(64, return_sequences=False))(x)
    x = Dropout(0.3)(x)

    # [Step C] Auxiliary Inputs 병합
    # 에너지 값과 tRNA 점수를 각각 처리 후 메인 서열 정보와 합침
    y = Dense(16, activation='relu')(input_energy)
    z = Dense(16, activation='relu')(input_trna)

    # 모든 정보를 하나로 융합 (Concatenate)
    merged = Concatenate()([x, y, z])

    # 공유된 최종 특징 벡터
    shared_dense = Dense(128, activation='relu')(merged)

    # ---------------------------------------------------------
    # 3. Multi-Heads (개별 예측 헤드)
    # ---------------------------------------------------------
    outputs = []

    for t_name in target_names:
        # Layer 이름에 특수문자가 들어가지 않도록 정제 (예: SpCas9-NG -> SpCas9_NG)
        clean_name = re.sub(r'[^0-9a-zA-Z_]', '_', t_name)

        # 각 단백질 전용 신경망 (Independent Logic)
        h = Dense(64, activation='relu', name=f'Dense_{clean_name}')(shared_dense)
        h = Dropout(0.2)(h)

        # 최종 출력: 0~100 사이의 효율 점수 예측 (Linear Activation)
        out = Dense(1, activation='linear', name=f'Output_{clean_name}')(h)
        outputs.append(out)

    # ---------------------------------------------------------
    # 4. 모델 생성
    # ---------------------------------------------------------
    model = Model(inputs=[input_seq, input_energy, input_trna], outputs=outputs)
    return model

# 실행 파트

# ---------------------------------------------------------
# 1. 타겟 리스트 수정 (대소문자 통일!)
# ---------------------------------------------------------
targets = [
    'SpCas9', 'SpCas9-NG', 'VRQR variant', 'xCas', 'Sniper-Cas9',
    'SpCas9-HF.1', 'eSpCas9(1.1)', 'HypaCas9', 'evoCas9',
    'Cas12a'  # 🚨 수정: 'cas12a' -> 'Cas12a' (대문자 C)
]

# 더미 데이터 (에러 방지용)
X_energy_dummy = np.zeros((len(X), 1))
X_trna_dummy = np.zeros((len(X), 1))

# ---------------------------------------------------------
# 2. 딕셔너리로 데이터 매핑 (논리 수정됨)
# ---------------------------------------------------------
train_Y_dict = {}
train_weight_dict = {}
loss_dict = {}
metrics_dict = {}

print("🔄 데이터 딕셔너리 생성 중...")

# 가중치 미리 계산 (Type_list 변수가 있어야 함)
sp_weight = (Types == 0).astype(float)
cpf1_weight = (Types == 1).astype(float)

# 나중에 리스트로 변환하기 위해 순서를 기억할 리스트
ordered_keys = []

for i, t_name in enumerate(targets):
    clean_name = re.sub(r'[^0-9a-zA-Z_]', '_', t_name)
    key_name = f'Output_{clean_name}'
    ordered_keys.append(key_name) # 순서 저장

    # 1) 정답(Y) 매핑
    train_Y_dict[key_name] = Y[:, i]

    # 2) Loss & Metric
    loss_dict[key_name] = 'mse'
    metrics_dict[key_name] = 'mae'

    # 3) 가중치(Weight) 매핑 - 🚨 대소문자 수정으로 이제 정상 작동함
    if t_name == 'Cas12a': # 이제 targets에 'Cas12a'가 있으므로 True가 됨!
        train_weight_dict[key_name] = cpf1_weight
    else:
        train_weight_dict[key_name] = sp_weight

print(f"✅ 딕셔너리 준비 완료! (Cas12a 가중치 로직 수정됨)")

# ---------------------------------------------------------
# 3. 딕셔너리 -> 리스트 변환 (KeyError 완벽 차단)
# ---------------------------------------------------------
# Keras 에러를 피하기 위해, 학습 직전에 리스트로 바꿉니다.
train_Y_list = [train_Y_dict[k] for k in ordered_keys]
train_weight_list = [train_weight_dict[k] for k in ordered_keys]
loss_list = [loss_dict[k] for k in ordered_keys]
metrics_list = [metrics_dict[k] for k in ordered_keys]

# ---------------------------------------------------------
# 4. 모델 학습 시작
# ---------------------------------------------------------
model = build_universal_hydra_model(targets)

# compile에도 리스트를 넣습니다.
model.compile(optimizer=Adam(0.001), loss=loss_list, metrics=metrics_list)

print(f"\n🚀 Universal 10-Head Model 최종 학습 시작!")
history = model.fit(
    x=[X, X_energy_dummy, X_trna_dummy],
    y=train_Y_list,              # 리스트로 입력 (안전)
    sample_weight=train_weight_list, # 리스트로 입력 (안전)
    batch_size=64,
    epochs=20,
    validation_split=0.2,
    verbose=1
)

model.save('universal_10_hydra_final.keras')
print("🎉 학습 완료 및 모델 저장 성공!")

🔄 데이터 딕셔너리 생성 중...
✅ 딕셔너리 준비 완료! (Cas12a 가중치 로직 수정됨)

🚀 Universal 10-Head Model 최종 학습 시작!
Epoch 1/20
690/690 ━━━━━━━━━━━━━━━━━━━━ 91s 109ms/step - Output_Cas12a_loss: 0.3303 - Output_Cas12a_mae: 1.0860 - Output_HypaCas9_loss: 211.8395 - Output_HypaCas9_mae: 9.9466 - Output_Sniper_Cas9_loss: 284.8977 - Output_Sniper_Cas9_mae: 11.9958 - Output_SpCas9_HF_1_loss: 240.7170 - Output_SpCas9_HF_1_mae: 10.5645 - Output_SpCas9_NG_loss: 294.6096 - Output_SpCas9_NG_mae: 14.2900 - Output_SpCas9_loss: 317.6978 - Output_SpCas9_mae: 12.6781 - Output_VRQR_variant_loss: 232.1984 - Output_VRQR_variant_mae: 11.9963 - Output_eSpCas9_1_1__loss: 247.3857 - Output_eSpCas9_1_1__mae: 10.9259 - Output_evoCas9_loss: 102.9279 - Output_evoCas9_mae: 5.7319 - Output_xCas_loss: 210.1206 - Output_xCas_mae: 11.1034 - loss: 2142.7266 - val_Output_Cas12a_loss: 3.5295 - val_Output_Cas12a_mae: 1.2370 - val_Output_HypaCas9_loss: 0.0000e+00 - val_Output_HypaCas9_mae: 3.2929 - val_Output_Sniper_Cas9_loss: 0.0000e+00 - val_Outp